# Modelos Theta y 4Theta

# Modelos Theta

In [1]:
# 
import os, sys
from google.colab import drive
drive.mount('/content/mnt', force_remount=True)
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks/Librerias', nb_path)
#sys.path.insert(0, nb_path)  # or append(nb_path)
sys.path.append(nb_path)  # or append(nb_path)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Es necesario tener instalado DARTS

[Darts: https://unit8co.github.io/darts/index.html](https://unit8co.github.io/darts/index.html)


In [ ]:
#!pip install darts

In [ ]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

- Utilizamos la libreria **darts**
- Seleccionamos los datos de Lineas aereas

In [ ]:
# %%
from darts import TimeSeries
from darts.datasets import AirPassengersDataset


- **Grafico de la Serie Temporal de lineas aereas**



In [ ]:

# %%
series = AirPassengersDataset().load()
series


In [ ]:
#import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,6)
series.plot()

- Diferentes formas de seleccionat TRAIN y TEST



```
split_before
```



In [ ]:
# %%
series1, series2 = series.split_before(0.75)
series1.plot()
series2.plot()



- Con el número de observaciones

In [ ]:

# %%
series1, series2 = series[:-36], series[-36:]
series1.plot()
series2.plot()


Con la fecha

In [ ]:
train, val = series.split_before(pd.Timestamp("19580101"))
train.plot(label="training")
val.plot(label="validation")

Comenzamos con el modelo THETA

In [ ]:

# %%
from darts.models import Theta, FourTheta
from darts.metrics import mape

model =Theta(seasonality_period=12) # Supone Theta 2
model.fit(train)
forecast = model.predict(len(val))
print("model {} obtains MAPE: {:.2f}%".format(model, mape(val, forecast)))




In [ ]:
# %%
train.plot(label="train")
val.plot(label="true")
forecast.plot(label="prediction")

In [ ]:

# %%
# Search for the best theta parameter, by trying 50 different values
thetas = 2- np.linspace(-10, 10, 50)

best_mape = float("inf")
best_theta = 0




for theta in thetas:

    model = Theta(theta)
    model.fit(train)
    pred_theta = model.predict(len(val))
    res = mape(val, pred_theta)

    if res < best_mape:
        best_mape = res
        best_theta = theta


In [ ]:
thetas

In [ ]:
best_theta

In [ ]:

# %%
best_theta_model = Theta(best_theta)
best_theta_model.fit(train)
pred_best_theta = best_theta_model.predict(len(val))

print(
    "The MAPE is: {:.2f}, with theta = {}.".format(
        mape(val, pred_best_theta), best_theta
    )
)


In [ ]:
# %%
train.plot(label="train")
val.plot(label="true")
pred_best_theta.plot(label="prediction")

In [ ]:
best_theta_model.model_params

# Modelo 4Theta

In [ ]:
model =FourTheta(seasonality_period=12)
model.fit(train)
forecast = model.predict(len(val))
print("model {} obtains MAPE: {:.2f}%".format(model, mape(val, forecast)))




In [ ]:
# %%
train.plot(label="train")
val.plot(label="true")
forecast.plot(label="prediction")

In [ ]:
from darts.utils.utils import SeasonalityMode, TrendMode, ModelMode
theta_grid = {
    #'theta':2- np.linspace(-10, 10, 10),
    'theta':[-4,-3,-2,-1,1,2,3,4],
    'model_mode': [ModelMode.ADDITIVE,ModelMode.MULTIPLICATIVE],
    'season_mode': [SeasonalityMode.MULTIPLICATIVE,SeasonalityMode.ADDITIVE],
    'trend_mode': [TrendMode.EXPONENTIAL,TrendMode.LINEAR]
}

best_grid_model=FourTheta.gridsearch(parameters=theta_grid,
                                series=train,
                                forecast_horizon=36, # 12
                                start=0.5,
                                last_points_only=False,
                                metric=mape,
                                reduction=np.mean,
                                verbose=True,
                                n_jobs=-1)

best_grid_model

In [ ]:
modelo=FourTheta(theta=best_grid_model[1]['theta'],
                 model_mode=ModelMode.ADDITIVE,
                 season_mode=SeasonalityMode.MULTIPLICATIVE,
                 trend_mode=TrendMode.LINEAR
                 )
modelo.fit(train)
pred_modelo = modelo.predict(len(val))

train.plot(label="train")
val.plot(label="true")
pred_modelo.plot(label="prediction")

In [ ]:
print("model {} obtains MAPE: {:.2f}%".format(modelo, mape(val, pred_modelo)))

In [ ]:

# %%
train.plot(label="train")
val.plot(label="true")
pred_modelo.plot(label="prediction 4Theta")
pred_best_theta.plot(label="prediction Theta")

#MANU: 


# Cross Validation: Historical

In [ ]:


# %%
historical_fcast_theta = best_theta_model.historical_forecasts(
    series, start=0.5, forecast_horizon=36, verbose=True, stride=1
)

series.plot(label="data")
historical_fcast_theta.plot(label="backtest 12-months ahead forecast (Theta)")
print("MAPE = {:.2f}%".format(mape(historical_fcast_theta, series)))




## BackTest

In [ ]:

# %%


raw_errors = best_theta_model.backtest(
    series, 
    start=0.4, 
    forecast_horizon=12, 
    metric=mape, 
    reduction=None, # None: return errors
    verbose=True
)

from darts.utils.statistics import plot_hist

plot_hist(
    raw_errors,
    bins=np.arange(0, max(raw_errors), 1),
    title="Individual backtest error scores (histogram)",
)



In [ ]:

# %%
average_error = best_theta_model.backtest(
    series,
    start=0.4,
    forecast_horizon=12,
    metric=mape,
    reduction=np.mean,  # this is actually the default
    verbose=True,
)

print("Average error (MAPE) over all historical forecasts: %.2f" % average_error)




In [ ]:
average_error

In [ ]:
raw_errors